In [1]:
!pip install pycapnp

You should consider upgrading via the '/Users/madhavajay/dev/PySyft/.tox/syft.jupyter/bin/python -m pip install --upgrade pip' command.


In [2]:
%load_ext autoreload
%autoreload 2
# %load_ext line_profiler
import numpy as np
import pandas as pd
# import torch
# import gc
import syft as sy
from syft.core.adp.entity import Entity
from syft.core.adp.entity_list import DataSubjectList
# from pympler.asizeof import asizeof #pip install pympler
from syft import serialize
from syft import deserialize
import timeit
# from syft.core.tensor.autodp.dp_tensor_converter import convert_to_gamma_tensor

# from functools import reduce
import time
import pyarrow as pa
from syft.core.tensor.autodp.row_entity_phi import RowEntityPhiTensor as REPT

In [3]:
df = pd.read_parquet("1M_rows_dataset_sample.parquet")
print("Number of Rows: ",df.shape[0])
df.head()
print(df.shape[0])

Number of Rows:  1000000
1000000


In [4]:
# factor = 1
# df = pd.concat([df] * factor, ignore_index=True)

In [5]:
# df.shape[0] / 1e6

In [6]:
# # Let's login into the domain node
# domain_node = sy.login(email="info@openmined.org", password="changethis", port=8081)

In [7]:
%%time
name = "Tweets- 100000 rows dataset "
impressions = ((np.array(list(df['impressions'])))).astype(np.int32)
publication_title = ((list(df['publication_title'])))

CPU times: user 246 ms, sys: 23.6 ms, total: 270 ms
Wall time: 270 ms


In [8]:
type(publication_title[0])

str

In [9]:
%%time
entities = list()
for i in range(len(publication_title)):
    entities.append(Entity(name=publication_title[i]))

CPU times: user 1.5 s, sys: 35.9 ms, total: 1.54 s
Wall time: 1.55 s


In [10]:
t0 = time.time()
tweets_data = sy.Tensor(impressions).private(min_val=0, max_val=30, entities = entities)
tf = time.time() - t0
print(tf)

7.180011034011841


In [11]:
def extend_tweet_data(data, multiplier: int):
    new_data = data.copy()
    new_data.child.child = new_data.child.child.repeat(multiplier)
    new_data.child._entities = DataSubjectList.from_objs(np.array(["ishan"] * multiplier * len(new_data.child._entities.entities_indexed)))
    return new_data

In [51]:
def benchmark(data):
    start = time.time()
    res = data.child.arrow_serialize()
    end = time.time()
    print(end-start)
    return res

In [52]:
mil = tweets_data

In [53]:
len(mil)

1000000

In [54]:
b = benchmark(mil)

step 1: 0.02001500129699707
Total Size:  0.15044795849609374
step 2: 0.00023818016052246094
Capnp Size:  0.1436920166015625
0.021291017532348633


In [55]:
type(b)

bytes

In [70]:
x = REPT.arrow_deserialize(b)

child_metadata 4000192 int32
step 2: 0.10100126266479492


In [71]:
x

RowEntityPhiTensor(child=[9029 9029 9029 ... 9709 9709 9709])

In [72]:
len(mil.child)

1000000

In [73]:
len(x.child)

1000000

In [74]:
(mil.child.child == x.child).all()

True

In [75]:
mil.child._min_vals == x._min_vals

True

In [76]:
mil.child._max_vals == x._max_vals

True

In [77]:
mil.child._entities.entities_indexed == x._entities.entities_indexed

array([ True,  True,  True, ...,  True,  True,  True])

In [78]:
mil.child._entities.one_hot_lookup == x._entities.one_hot_lookup

array([ True,  True,  True])

In [79]:
mil.child._entities.one_hot_lookup == x._entities.one_hot_lookup

array([ True,  True,  True])

In [80]:
mil.child._entities.one_hot_lookup

array([<Entity:Al Jazeera>, <Entity:AlterNet>,
       <Entity:American Greatness>], dtype=object)

In [81]:
x._entities.one_hot_lookup

array(['Al Jazeera', 'AlterNet', 'American Greatness'], dtype='<U18')

In [83]:
ten_mil = extend_tweet_data(mil, 10)

In [84]:
b = benchmark(ten_mil)

step 1: 0.10651898384094238
Total Size:  0.00719995849609375
step 2: 0.0001380443572998047
Capnp Size:  0.00701141357421875
0.1077120304107666


In [82]:
hundred_mil = extend_tweet_data(mil, 100)

In [86]:
len(hundred_mil)

100000000

In [85]:
b = benchmark(hundred_mil)

step 1: 1.0342206954956055
Total Size:  0.06212495849609375
step 2: 0.0001811981201171875
Capnp Size:  0.05941009521484375
1.0353949069976807


In [ ]:
# pyarrow record batch
# benchmark(hundred_mil)
# step 1: 11.958800077438354
# Total Size:  900.042533
# step 2: 0.91678786277771
# 13.002313137054443

In [ ]:
benchmark(hundred_mil)

In [ ]:
bill = extend_tweet_data(mil, 1000)

In [ ]:
benchmark(bill)

In [ ]:
# len(tweets_data.child.child)/1e6

In [ ]:
# len(tweets_data.child._entities)/1e6

In [ ]:
# %lprun -f tweets_data.child.child[0].entity.simple_assets_for_serde tweets_data.child.child[0].entity.simple_assets_for_serde()

In [ ]:
# %lprun -f tweets_data.child.arrow_serialize tweets_data.child.arrow_serialize()

In [ ]:
benchmark(bill)

In [ ]:
# tweets_data.child.child[0].scalar_manager.prime2symbol

In [ ]:
%%time
pa.serialize(np.random.rand(1_000_000))

In [ ]:
%%time
for i in range(1_000_000):
    pa.serialize(np.random.rand(1))

In [ ]:
%%time
serialize(tweets_data.child.child[0],to_bytes=True)

In [ ]:
%%time
result = tweets_data.sum()

In [ ]:
tweets_data.child.serde_concurrency= 1

In [ ]:
#measuring size of objects in python is not easy, 
#as there might be several references in a user defined class
#we pympler library to measure the size of an object.
#pymler does not give an exact size on disk, but a close accurate instead of sys.getsizeof()

print("Size of Twitter Tensor Data(MB) ====> " , asizeof(tweets_data)/(1024*1024)) 

In [ ]:
#Twitter data serialization.
start = timeit.default_timer()
serialized_data = serialize(tweets_data,to_bytes=True)
end = timeit.default_timer()
print("Serialization Time =====>" , end-start," seconds" )
print("Twitter Serialized Data Size ",len(serialized_data)/(1024*1024))


In [ ]:
%lprun -f tweets_data.child._object2proto tweets_data.child._object2proto()

In [ ]:
#Twitter data derserialization
start = timeit.default_timer()
deserialized_data = deserialize(serialized_data,from_bytes=True)
end = timeit.default_timer()
print("Deserialization Time =====>" , end-start," seconds" )
print("Twitter Deserialized Data Size ",asizeof(deserialized_data)/(1024*1024))


In [ ]:
%%time
#Uploading Dataset....
domain_node.load_dataset(
assets={"tweets": tweets_data},
name=name,
description="""Tweets- 1M rows """
)

#The main bottleneck is the serialization as uploading dataset is a blocking call(immediate_msg_with_reply), 
#bottleneck = serialization at client + DeSerialization at node


In [ ]:
domain_node.datasets

In [ ]:
dataset = domain_node.datasets[0]["tweets"]

In [ ]:
%%time
res = dataset.sum()
res.block

In [ ]:
res.exists

In [ ]:
res = res.publish(sigma=0.1)

In [ ]:
res.get()

In [ ]:
l = tweets_data.child.child

In [ ]:
a = l[0]
b= l[50611]
c = l[83931]
print(a)
print(b)
print(c)

In [ ]:
split_lst=[]
d = {}
c=0
for i in l:
    if i.entity not in d:
        d[i.entity]=c
        split_lst.append([i])
        c+=1
    else:
        split_lst[d[i.entity]].append(i)
        

In [ ]:
first= split_lst[0]
second = split_lst[1]
third = split_lst[2]
fourth = split_lst[3]


In [ ]:
def list_sum(lst):
    s=lst[0]
    for i in range(1,len(lst)):
        s+=lst[i]
    return s

In [ ]:
s=first[0]
for i in first[1::]:
    s=s+i
print("final: ",s)
    

In [ ]:
def lst_sum(a,b):
    return a+b

In [ ]:
t1 = reduce(lst_sum,first)
t2 = reduce(lst_sum,second)
t3 = reduce(lst_sum,third)
t4 = reduce(lst_sum,fourth)

In [ ]:
%%time
r1 = t1+t2
r2 = t3+t4

In [ ]:
tweets_data.child.child[1].dtype

In [ ]:
from syft.core.adp.entity import Entity

In [ ]:
a = Entity("a")
b = Entity("a")

In [ ]:
a == b

In [ ]:
a.id

In [ ]:
b.id

In [ ]:
a = ["a", "b", "c"]

In [ ]:
x = np.array(a)

In [ ]:
x

In [ ]:
y = pa.array(x)

In [ ]:
y

In [ ]:
x

In [ ]:
apache_arrow = pa.Tensor.from_numpy(obj=x)

In [ ]:
rb = pa.RecordBatch.from_pylist([{"entity":x}])

In [ ]:
rb

In [ ]:
y = rb[0].to_numpy(zero_copy_only=False)

In [ ]:
y